## Inportaciones

In [11]:
import pyodbc
import pandas as pd
import sqlite3
import datetime

Se debe cambiar a la dirección correspondiente

In [12]:
db_path = r"X:\Instituto\BI\Proyecto-Capstone\POS\BackEnd\db.sqlite3"

## Conexión

In [13]:
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=POSREF;"
    "Trusted_Connection=yes;"
)

# Conexión
try:
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa.")
except Exception as e:
    print("Error de conexión:", e)

Conexión exitosa.


Ma_familia 1 son platos y menus 

Ma_familia 2 son papas

Ma_familia 3 son bebidas

Ma_familia 4 son descuentos (Los remplace por extras)

## Extract de Ventas

In [14]:

query = """
SELECT 
    a.Cp_cliente as clienteComp1,
    a.Cp_total- a.Cp_vuelto as totalComp1,
    a.Cp_fecha as fechaComp1,
    a.Cp_hora as horaComp1,
    a.Cp_pagado as pagadoComp1,
    b.Cp1_tipdoc as tipodocComp2


FROM COMP1 AS a

LEFT JOIN COMP2 AS b
    ON  b.cp_folio = a.cp_folio
    AND b.Ma_Emp   = a.Ma_Emp

"""

df = pd.read_sql(query, conn)
print("Filas leídas:", len(df))
df.head(20)

Filas leídas: 536


C:\Users\sherr\AppData\Local\Temp\ipykernel_26340\3022905162.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,clienteComp1,totalComp1,fechaComp1,horaComp1,pagadoComp1,tipodocComp2
0,1.0,21490.0,2025-11-07,12:07:44,S,EF
1,1.0,14990.0,2025-11-12,22:03:14,S,EF
2,1.0,13490.0,2025-11-12,22:04:44,S,EF
3,1.0,4900.0,2025-11-13,12:45:52,S,EF
4,1.0,25290.0,2025-11-13,12:46:30,S,TD
5,1.0,31190.0,2025-11-13,13:07:24,S,TD
6,1.0,3900.0,2025-11-13,13:19:03,S,EF
7,1.0,5990.0,2025-11-13,13:28:24,S,TD
8,1.0,4500.0,2025-11-13,13:32:09,S,TD
9,1.0,11490.0,2025-11-13,13:33:10,S,EF


## Transformaciones

In [15]:
df["fechaComp1"] = pd.to_datetime(df["fechaComp1"])
df["horaComp1"] = pd.to_timedelta(df["horaComp1"].astype(str))


df["Fecha"] = df["fechaComp1"] + df["horaComp1"]
df= df.drop(columns=['fechaComp1', 'horaComp1'])

In [16]:
df["clienteComp1"]= df["clienteComp1"].astype(int)
df["totalComp1"]= df["totalComp1"].astype(int)

In [17]:
df.head(10)

,clienteComp1,totalComp1,pagadoComp1,tipodocComp2,Fecha
0,1,21490,S,EF,2025-11-07 12:07:44
1,1,14990,S,EF,2025-11-12 22:03:14
2,1,13490,S,EF,2025-11-12 22:04:44
3,1,4900,S,EF,2025-11-13 12:45:52
4,1,25290,S,TD,2025-11-13 12:46:30
5,1,31190,S,TD,2025-11-13 13:07:24
6,1,3900,S,EF,2025-11-13 13:19:03
7,1,5990,S,TD,2025-11-13 13:28:24
8,1,4500,S,TD,2025-11-13 13:32:09
9,1,11490,S,EF,2025-11-13 13:33:10


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   clienteComp1  536 non-null    int32         
 1   totalComp1    536 non-null    int32         
 2   pagadoComp1   536 non-null    object        
 3   tipodocComp2  536 non-null    object        
 4   Fecha         536 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int32(2), object(2)
memory usage: 16.9+ KB


In [19]:
df["subtotal"] = round(df["totalComp1"] * 0.81,0)
df["Iva"] = round(df["totalComp1"] * 0.19,0)

In [20]:
df["subtotal"]= df["subtotal"].astype(int)
df["Iva"]= df["Iva"].astype(int)

In [21]:
df.loc[
    df["tipodocComp2"].str.contains("TD", case=False, na=False),
    "tipodocComp2"
] = 1

df.loc[
    df["tipodocComp2"].str.contains("EF", case=False, na=False),
    "tipodocComp2"
] = 2
df.loc[
    df["tipodocComp2"].str.contains("TC", case=False, na=False),
    "tipodocComp2"
] = 5

df.loc[
    df["tipodocComp2"].str.contains("CC", case=False, na=False),
    "tipodocComp2"
] = 6

In [22]:
df.loc[
    df["pagadoComp1"].str.contains("S", case=False, na=False),
    "pagadoComp1"
] = 2

In [23]:
df["tipodocComp2"]= df["tipodocComp2"].astype(int)
df["pagadoComp1"]= df["pagadoComp1"].astype(int)

## Creación de tabla SesionCaja

In [24]:
df["SesionCaja"] = 24
df["Usuario"] = 21


In [27]:
df

,clienteComp1,totalComp1,pagadoComp1,tipodocComp2,Fecha,subtotal,Iva,SesionCaja,Usuario
0,1,21490,2,2,2025-11-07 12:07:44,17407,4083,24,21
1,1,14990,2,2,2025-11-12 22:03:14,12142,2848,24,21
2,1,13490,2,2,2025-11-12 22:04:44,10927,2563,24,21
3,1,4900,2,2,2025-11-13 12:45:52,3969,931,24,21
4,1,25290,2,1,2025-11-13 12:46:30,20485,4805,24,21
...,...,...,...,...,...,...,...,...,...
531,1,2900,2,1,2025-11-18 20:33:52,2349,551,24,21
532,1,13790,2,2,2025-11-18 20:41:47,11170,2620,24,21
533,1,14700,2,1,2025-11-18 20:51:16,11907,2793,24,21
534,1,7990,2,2,2025-11-18 20:51:58,6472,1518,24,21


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   clienteComp1  536 non-null    int32         
 1   totalComp1    536 non-null    int32         
 2   pagadoComp1   536 non-null    int32         
 3   tipodocComp2  536 non-null    int32         
 4   Fecha         536 non-null    datetime64[ns]
 5   subtotal      536 non-null    int32         
 6   Iva           536 non-null    int32         
 7   SesionCaja    536 non-null    int64         
 8   Usuario       536 non-null    int64         
dtypes: datetime64[ns](1), int32(6), int64(2)
memory usage: 25.3 KB


## Inicialización de la base de datos sqlite 3 insersión de datos

In [25]:
conn_sqlite = sqlite3.connect(db_path)
cursor = conn_sqlite.cursor()

In [35]:
rows = []
for _, row in df.iterrows():
    rows.append((
        int(row["clienteComp1"]),          
        int(row["totalComp1"]),          
        int(row["pagadoComp1"]),            
        float(row["tipodocComp2"]),       
        str(row["Fecha"]),           
        int(row["subtotal"]),
        int(row["Iva"]),
        int(row["SesionCaja"]),
        int(row["Usuario"])    
    ))

In [36]:
insert_sql = """
INSERT INTO venta_venta
(cliente_id, total, estado_id, metodo_pago_id, fecha, subtotal,iva,sesion_caja_id,usuario_id)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

In [37]:
try:
    cursor.executemany(insert_sql, rows)
    conn_sqlite.commit()
    conn_sqlite.close()
    print("Carga a inventario_producto completada.")
except Exception as e:
    print(f"Error durante la carga a inventario_producto: {e}")

Carga a inventario_producto completada.
